In [ ]:
# This cell just imports relevant modules

import numpy as np
import pylab
from math import pi
from sympy import sin, cos, Function, Symbol, diff, integrate, matrices
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
###### TRANSFORMATION MATRICES ######
###### Lecture 5, slide 9 ######

# Define the transformation matrix in Python using numpy.matrix
mD = np.matrix([[1.25, 0],
                  [0, 0.8]])
           
# A list of coordinate vectors in the form [x,y]. 
# These are stored as numpy arrays so we can easily multiply them 
# by the transformation matrix.
vCoordinates = [np.array([1, 0]),
               np.array([0, 1]),
               np.array([-1, 0]),
               np.array([0, -1])]

# Take each coordinate and transform it
for i in range(0, len(vCoordinates)):
    # We need to reshape the array so it is conformable
    # (i.e. it is of the right dimension for matrix-vector
    # multiplication). In this case, we need it to be 2 x 1.
    vCoordinates[i] = np.reshape(vCoordinates[i], (2,1))
    print(mD*vCoordinates[i]) 
   
sDet = np.linalg.det(mD)
sVolStrain = sDet - 1 # The volumetric strain

print("Determinant of D is: %f" % sDet) 
print("This implies:") 
if(sVolStrain == 1):
    print("No volume change") 
elif(sVolStrain > 1):
    print("Increase in volume") 
elif(sVolStrain > 0 and sVolStrain < 1):
    print("Decrease in volume") 
else:
    print("No geological meaning") 

In [ ]:
###### EIGENVALUES OF A 2 x 2 MATRIX ######
###### Lecture 5, slide 18 ######
mA = np.matrix([[1,4],
                  [1,1]]) 
                  
# A simple way of finding eigenvalues is to use 
# the eigvals function.
sEigValues = np.linalg.eigvals(mA)
print("The eigenvalues of mA are: ", sEigValues) 
             
# Alternatively, we could work out the characteristic polynomial
# for lambda, and then find the roots.
# Computes det(mA - sLambda*mI) and returns a list of scalar coefficients [sX,sY,sZ] for 
# the characteristic polynomial sX*sLambda**2 + sY*sLambda + sZ
sCharacteristicPoly = np.poly(mA)
print("The characteristic polynomial of mA is: (%f*lambda**2) + (%f*lambda) + (%f)" 
      % (sCharacteristicPoly[0], sCharacteristicPoly[1], sCharacteristicPoly[2])) 
# Finds the roots (in this case, these will be the eigenvalues)
sRoots = np.roots(sCharacteristicPoly)
print("The roots of the characteristic polynomial are: ", sRoots) 

In [ ]:
###### EIGENVECTORS OF A 2 x 2 MATRIX ######
###### Lecture 5, slide 20 ######
mA = np.matrix([[1,4],
                  [1,1]]) 
                  
# A simple way of finding eigenvectors is to use 
# the eig function. Note that this gives BOTH the eigenvalues in an array (say 'sEigValues')
# AND eigenvectors in a matrix data type (say 'vEigVectors'). 
#The i-th eigenvector, stored in the column vEigVectors[:,i], 
# corresponds to the eigenvalue stored in sEigValues[i].
(sEigValues, vEigVectors) = np.linalg.eig(mA)
for i in range(0, len(sEigValues)):
    print("Eigenvalue #", str(i+1), " is: ", sEigValues[i]) 
    print("The corresponding eigenvector is:\n", vEigVectors[:,i]) 

In [ ]:
# define function to plot original vectors & unit circle, and transformed vectors & unit circle

def plot_ellipse(a, b, c, d, horizontal_scale, vertical_scale):
    m = np.array([[a, b], [c, d]]) # matrix
    
    (sEigValues, vEigVectors) = np.linalg.eig(m) 
    
    # print eigenvalues
    print("Eigenvalue 1 = %.f" % (sEigValues[0]))
    print("Eigenvalue 2 = %.f" % (sEigValues[1]), "\n")
    
    # print points on unit circle, coordinates = eigenvectors
    point1 = np.array([vEigVectors[0][0], vEigVectors[1][0]]) 
    print("Eigenvector 1 =", point1)
    point2 = np.array([vEigVectors[0][1], vEigVectors[1][1]])
    print("Eigenvector 2 =", point2, "\n")
    
    # transformed points
    transformed_point1 = m @ point1
    print("New vector 1 =", transformed_point1)
    tp1 = sEigValues[0]*point1
    print("Eigenvalue 1 * eigenvector 1\n= %.f" % (sEigValues[0]), "*", point1, "=", tp1, "\n")
    transformed_point2 = m @ point2
    print("New vector 2 =", transformed_point2)
    tp2 = sEigValues[1]*point2
    print("Eigenvalue 2 * eigenvector 2\n= %.f" % (sEigValues[1]), "*", point2, "=", tp2, "\n")
    
    # prove new vector = eigenvalue * eigenvector
    if np.allclose(transformed_point1, tp1) == True and np.allclose(transformed_point1, tp1) == np.allclose(transformed_point2, tp2):
        print("New vector = eigenvalue * eigenvector \n")
    else:
        print("Error \n")
    
    # plot ellipse
    u = 0  # x-position of center
    v = 0  # y-position of center
    a = abs(sEigValues[0])  # radius on x-axis
    b = abs(sEigValues[1])  # radius on y-axis
    t_rot = np.arctan(vEigVectors[1][0]/vEigVectors[0][0]) # rotation angle
    
    t = np.linspace(0, 2*pi, 100)
    
    Ell = np.array([a*np.cos(t), b*np.sin(t)]) # ellipse coordinates
    R_rot = np.array([[cos(t_rot), -sin(t_rot)],[sin(t_rot), cos(t_rot)]]) # 2D rotation matrix
    Ell_rot = np.zeros((2, Ell.shape[1]))
    for i in range(Ell.shape[1]):
        Ell_rot[:,i] = R_rot @ Ell[:,i] # rotated ellipse coordinates
        
    ang = np.allclose(point1@point2, 0)
    if ang == True:
        print("The matrix is symmetrical, so the two eigenvectors are perpendicular")
    else:
        print("The matrix is NOT symmetrical, so the two eigenvectors are NOT perpendicular")
        
    plt.figure(figsize=(horizontal_scale,vertical_scale))
    plt.plot([0, point1[0]], [0, point1[1]], 'r', label='eigenvector1', linewidth=5)
    plt.plot([0, point2[0]], [0, point2[1]], 'c', label='eigenvector2', linewidth=5)
    plt.plot(np.cos(t), np.sin(t), 'k', label='unit circle')
    # new vector = eigenvalue * eigenvector
    plt.plot([0, tp1[0]], [0, tp1[1]], 'k--', label='new vector 1')
    plt.plot([0, tp2[0]], [0, tp2[1]], 'b--', label='new vector 2')
    plt.plot(u+Ell_rot[0,:], v+Ell_rot[1,:], 'g', label='transformed unit circle')
    plt.legend(loc='upper left', fontsize=10)
    plt.title('The meaning of eigenvectors and eigenvalues illustrated on a unit circle', fontsize=14)
    plt.grid(True)
    plt.show()

In [ ]:
# plot solution to Lecture 5 Slide 20
plot_ellipse(1,4,1,1,10,6)

In [ ]:
###### REPEATED EIGENVALUES ######
###### Lecture 5, slide 24 ######
mA = np.matrix([[1,0],
                  [0,1]]) 
#NOTE: Here we could also define mA using numpy.identity(2).

#NOTE: NumPy will give ALL eigenvalues, including the repeated ones.
(sEigValues, vEigVectors) = np.linalg.eig(mA)
for i in range(0, len(sEigValues)):
    print("Eigenvalue #", str(i+1), " is: ", sEigValues[i]) 
    print("The corresponding eigenvector is:\n", vEigVectors[:,i]) 

In [ ]:
# plot solution to lecture 5 slide 24
plot_ellipse(1, 0, 0, 1, 6, 6)

In [ ]:
###### REAL AND COMPLEX EIGENVALUES ######
###### Lecture 5, slide 28 ######
mA = np.matrix([[0,1],
                  [-1,0]]) 
# NumPy prints out complex numbers in the form c = a + bj, where j is the imaginary number.
# We can use numpy.real(c) and numpy.imag(c) to print the real and imaginary parts respectively.
sEigValues = np.linalg.eigvals(mA) 
print("The eigenvalues of A are: ", sEigValues) 
print("The real part of the first eigenvalue is: %f" % np.real(sEigValues[0])) 
print("The imaginary part of the first eigenvalue is: %f" % np.imag(sEigValues[0])) 

In [ ]:
###### EXAMPLE EIGENVALUE PROBLEM ######
###### Lecture 5, slide 30 ######
mM = np.matrix([[3,-1],
                  [-1,3]])
(sEigValues, vEigVectors) = np.linalg.eig(mA)
for i in range(0, len(sEigValues)):
    print("Eigenvalue #", str(i+1), " is: ", sEigValues[i]) 
    print("The corresponding eigenvector is:\n", vEigVectors[:,i]) 

In [ ]:
# plot solution to lecture 5 slide 30
plot_ellipse(3, -1, -1, 3, 8, 8)

In [ ]:
###### SYMMETRIC MATRICES ######
###### Lecture 5, slide 31 ######
mNonSymmetric = np.matrix([[1,4],
                              [1,1]])
                              
(sEigValues, vEigVectors) = np.linalg.eig(mNonSymmetric)
for i in range(0, len(sEigValues)):
    print("Eigenvalue #", str(i+1), " is: ", sEigValues[i]) 
    print("The corresponding eigenvector is:\n", vEigVectors[:,i]) 
# NOTE 1: Unfortunately, because referencing a column of the matrix vEigVectors
# also returns another matrix data type (essentially a 'sub-matrix' of vEigVectors),
# we cannot use the numpy.dot function (which only operates on vectors/1D arrays).
# Instead, we'll simply use numpy.transpose to perform the dot product instead.
# NOTE 2: could also use numpy.vdot - read the documentation for more info.
print("The dot product of the two eigenvectors is: ", float(np.transpose(vEigVectors[:,0])*vEigVectors[:,1])) 

mSymmetric = np.matrix([[3,-1],
                           [-1,3]])
                              
(sEigValues, vEigVectors) = np.linalg.eig(mSymmetric)
for i in range(0, len(sEigValues)):
    print("Eigenvalue #", str(i+1), " is: ", sEigValues[i]) 
    print("The corresponding eigenvector is:\n", vEigVectors[:,i]) 
# The dot product should be zero here, as the two eigenvectors are orthogonal
# for any symmetric matrix. 
print("The dot product of the two eigenvectors is: ", float(np.transpose(vEigVectors[:,0])*vEigVectors[:,1])) 

In [ ]:
###### EIGENVALUE PROBLEM FOR A 3 x 3 MATRIX ######
###### Lecture 5, slide 37 ######
mM = np.matrix([[2,2,1],
                   [1,3,1],
                   [1,2,2]])
(sEigValues, vEigVectors) = np.linalg.eig(mM)
for i in range(0, len(sEigValues)):
    print("Eigenvalue #", str(i+1), " is: ", sEigValues[i]) 
    print("The corresponding eigenvector is:\n", vEigVectors[:,i]) 
## NOTE: If the values of x1, x2 or x3 are 'free' (i.e. can be chosen arbitrarily to obtain
## an independent eigenvector), NumPy does not necessarily choose values of 0 or 1, which
## is why the eigenvectors printed out here are different from those in your notes.
## They still satisfy the equation (A - lambda*I)x = 0 and are still independent eigenvectors.